In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

# 벡터 DB : Chroma vs Pinecone
- Chroma : 인메모리 vector DB, 로컬메모리 vector DB
- Pinecone : 클라우드 vector DB
    (pinecone console에 api key 생성 -> .env(PINECONE_API_KEY등록))

# 0. 패키지 설치

In [ ]:
# %pip install -q pinecone-client langchain-pinecone

# 1. Knowledge Base 구성을 위한 데이터 생성

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./tax_docs/with_markdown.docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
len(document_list)

In [3]:
# embedding : openAI API text-embedding-3-large
# https://python.langchain.com/v0.2/docs/integrations/text_embedding/upstage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large"
#     model="embedding-query"
)

In [4]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone()
# 데이터를 처음 업로드할 때
index_name = "tax-index-markdown"
database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embeddings,
    index_name=index_name
)
# 업로드한 DB벡터 가져올때
# database = PineconeVectorStore(
#     embedding=embeddings, # 질문을 임베딩하여 유사도 검색
#     index_name=index_name
# )

C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: total: 12.6 s
Wall time: 1min 12s


# 2. 답변 생성 전 retriever 확인

In [5]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retriever = database.as_retriever(search_kwargs={'k':4})
retriever.invoke(query)

[Document(id='2fc35e3d-216a-48e6-9ede-8168ff3593e1', metadata={'source': './tax_docs/with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에

# 3. 제공되는 prompt를 활용하여 답변 생성

In [6]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [7]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever, # database.as_retriever()
    chain_type_kwargs={"prompt":prompt}
)

In [8]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
ai_message = qa_chain.invoke({'query':query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 624만원에 해당합니다. 이는 5,000만원 과세표준 금액에 대한 누진 세율(624만원 + (5,000만원 초과분의 24%))을 적용한 결과입니다. 따라서 5천만원 연봉 직장인의 예상 소득세는 약 624만원입니다.'}

In [12]:
# 사람을 나타내는 표현 -> 거주자로 변경
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
dictionary = ["사람을 나타내는 표현 -> 거주자"]
prompt = ChatPromptTemplate.from_template(f"""사용자의 질문을 보고, 우리의 사전으 ㄹ참고해서
사용자의 질문을 변경해 주세요. 만약 변경할 필요가 없을 경우, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경우에는 질문만 리턴해주세요.
사전 : {dictionary}
질문 : {{question}}
""")

In [15]:
dictionary_chain = prompt | llm | StrOutputParser()
dictionary_chain.invoke({"question":query})

'연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [17]:
new_chain = {"query":dictionary_chain} | qa_chain

In [18]:
new_chain.invoke({"question":query})

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 약 624만원입니다. 이는 5,000만원의 과세표준에 대해 624만원 + (초과금액의 24%) 세율을 적용한 결과입니다. 따라서 대략적인 세액은 624만원으로 볼 수 있습니다.'}